# Vehicle Track

## Task1: Car Detection on Main Street

Import Libraries

In [ ]:
!pip install opencv-python-headless 
!pip install pyttsx3
!pip install numpy
!pip install opencv-python
!pip install pypiwin32

import cv2
import numpy as np

Create functions to import the provided videos and output path settings for saving the processed video

In [ ]:
def initialize_video(video_path):
    """Initialize video capture and retrieve video properties."""
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        raise ValueError(f"Error: Cannot open video at {video_path}")

    fps = int(video.get(cv2.CAP_PROP_FPS))
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    print(f"Video loaded: {video_path}")
    print(f"FPS: {fps}, Width: {frame_width}, Height: {frame_height}, Total Frames: {total_frames}")

    return video, fps, frame_width, frame_height, total_frames

def initialize_video_writer(output_path, fps, frame_width, frame_height):
    """Initialize video writer for output video."""
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    if not output_video.isOpened():
        raise ValueError(f"Error: Cannot create output video at `{output_path}`")

    print(f"Output video writer initialized at `{output_path}`")
    return output_video

# Initialize video and output writer
video1_path = "/content/Traffic_Laramie_1.mp4"
output1_path = "/content/Traffic_Detection_Output.mp4"
video, fps, frame_width, frame_height, total_frames = initialize_video(video1_path)
output_video = initialize_video_writer(output1_path, fps, frame_width, frame_height)


Video loaded: /content/Traffic_Laramie_1.mp4
FPS: 25, Width: 1040, Height: 600, Total Frames: 4448
Output video writer initialized at `/content/Traffic_Detection_Output.mp4`


Create functions for **Frame Preprocessing** and **Background Subtraction** (MOG2) for car detection and tracking

In [ ]:
# Function to preprocess each frame
def preprocess_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred


# Function to apply frame differencing
def apply_frame_differencing(current_frame, initial_frame):
    frame_diff = cv2.absdiff(initial_frame, current_frame)
    _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)
    return thresh


# Initialize MOG2 background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=16, detectShadows=True)
# Function to apply MOG background subtractor
def apply_background_subtraction(frame):
    fg_mask = bg_subtractor.apply(frame)
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    cleaned_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    dilated_mask = cv2.dilate(cleaned_mask, kernel, iterations=2)
    return dilated_mask


# Function to detect and draw green bounding boxes around detected cars on the main street
def detect_cars(frame, mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Gets the height of the video frame
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Dynamic threshold based on frame size
    MIN_CONTOUR_AREA = (frame_width * frame_height) * 0.001

    # Loop through each detected contour
    for contour in contours:
       # Focus on big objects like cars
        if cv2.contourArea(contour) < MIN_CONTOUR_AREA:
            continue

        # Get the bounding box coordinates
        x, y, w, h = cv2.boundingRect(contour)

        # Focus only on the bottom half (main street area)
        if y > height / 2:
          # Green box for main street cars
          cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return frame

Preprocessing the video to detect cars on main street

In [ ]:
# Read the initial frame for frame differencing
ret, initial_frame = video.read()
if not ret:
    print("Error: Could not read the initial frame.")
    video.release()
    exit(1)

# Preprocess the initial frame (grayscale + blur)
initial_gray = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)
initial_blur = cv2.GaussianBlur(initial_gray, (5, 5), 0)
print("Initial frame read successfully.")


print("Starting video processing...")
while True:
    ret, frame = video.read()
    if not ret:
        break

    # Preprocess the current frame
    preprocessed_frame = preprocess_frame(frame)

    # Frame differencing
    frame_diff = apply_frame_differencing(preprocessed_frame, initial_blur)

    # Background subtraction (MOG2)
    foreground_mask = apply_background_subtraction(preprocessed_frame)

    # Combine frame differencing and MOG2 masks
    combined_mask = cv2.bitwise_or(frame_diff, foreground_mask)

    # Detect the cars on mainstreet and draw bounding boxes around detected cars
    processed_frame = detect_cars(frame, combined_mask)

    # Write processed frame to output video
    output_video.write(processed_frame)

    # Press 'q' to exit early
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

print("The car detection is done!")

# Release resources
video.release()
output_video.release()
cv2.destroyAllWindows()

Initial frame read successfully.
Starting video processing...
The car detection is done!


## Task2: Car Counting Towards City Center

In [ ]:
# Function to draw the detection zone to define the counting area
def draw_detection_zone(frame, rect_center, rect_size, angle):
    """Draw the rotated detection zone on the video frame."""
    # Create the rotated rectangle
    rotated_rect = (rect_center, rect_size, angle)
    box = cv2.boxPoints(rotated_rect).astype(int)

    # Draw the rotated detection zone
    cv2.polylines(frame, [box], isClosed=True, color=(255, 255, 0), thickness=2)
    cv2.putText(frame, "Detection Zone", (rect_center[0] - 40, rect_center[1] - 60),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

    return box


# Function to detect and count cars based on motion within the detection zone
def detect_and_count_cars(frame, mask, detection_zone_box):
    """Detect and count cars using contours and centroid tracking."""
    global tracked_cars, car_count
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    detected_centroids = []

    # Gets the height of the video frame
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Loop through each detected contour
    for contour in contours:
        # Focus on big objects like cars
        if cv2.contourArea(contour) < 2500:
            continue

        # Get the bounding box adn centroid coordinates
        x, y, w, h = cv2.boundingRect(contour)
        cX, cY = x + w // 2, y + h // 2

        # Focus only on the bottom half (main street area)
        # Draw bounding box and centroid
        if y > height / 2:
          cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
          cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)

        # Check if the centroid is inside the detection zone
        if cv2.pointPolygonTest(detection_zone_box, (cX, cY), False) >= 0:
            detected_centroids.append((cX, cY))

    # Update tracked cars and avoid duplicate counting
    new_tracked_cars = []
    for cX, cY in detected_centroids:
        found = False
        for car_x, car_y, frames_left in tracked_cars:
            if abs(cX - car_x) < 20 and abs(cY - car_y) < 20:
                new_tracked_cars.append((cX, cY, FRAME_TIMEOUT))
                found = True
                break

        if not found:
            car_count += 1
            print(f"Car counted! Total count: {car_count}")
            new_tracked_cars.append((cX, cY, FRAME_TIMEOUT))

    # Remove expired tracked cars
    tracked_cars = [(x, y, frames_left - 1) for x, y, frames_left in tracked_cars if frames_left > 0]
    tracked_cars.extend(new_tracked_cars)

    # Display car count
    cv2.putText(frame, f"Cars to City: {car_count}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return frame


def process_video(video, output_video, fps, total_frames):
    """Process video frames, detect cars, and calculate cars per minute."""
    # Read the initial frame for frame differencing
    ret, initial_frame = video.read()
    if not ret:
        print("Error: Could not read the initial frame.")
        video.release()
        exit(1)

    # Preprocess the initial frame (grayscale + blur)
    initial_gray = cv2.cvtColor(initial_frame, cv2.COLOR_BGR2GRAY)
    initial_blur = cv2.GaussianBlur(initial_gray, (5, 5), 0)
    print("Initial frame read successfully.")

    global car_count, tracked_cars, FRAME_TIMEOUT
    # Car tracking variables
    car_count = 0
    tracked_cars = []
    FRAME_TIMEOUT = 30

    # Detection zone parameters
    rect_center = (60, int(frame_height / 2 + 50))
    rect_size = (70, 85)
    angle = 10


    print("Starting video processing...")
    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Preprocess current frame
        preprocessed_frame = preprocess_frame(frame)

        # Apply frame differencing and background subtraction
        frame_diff = apply_frame_differencing(preprocessed_frame, initial_blur)
        foreground_mask = apply_background_subtraction(preprocessed_frame)
        combined_mask = cv2.bitwise_or(frame_diff, foreground_mask)

        # Draw detection zone and get its coordinates
        detection_zone_box = draw_detection_zone(frame, rect_center, rect_size, angle)

        # Detect and count cars
        processed_frame = detect_and_count_cars(frame, combined_mask, detection_zone_box)

        # Write processed frame to output video
        output_video.write(processed_frame)

        # Exit early if 'q' is pressed
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    print("Video processing complete....")

    # Calculate cars per minute
    video_duration_minutes = (total_frames / fps) / 60
    cars_per_minute = car_count / video_duration_minutes if video_duration_minutes > 0 else 0.0

    # Final output
    print(f"\n----- Final Results -----")
    print(f"Total cars counted passing to the city center: {car_count}")
    print(f"Cars per minute: {cars_per_minute:.2f}")
    print("Video processing complete.")

    # Release resources
    video.release()
    output_video.release()
    cv2.destroyAllWindows()

In [ ]:
# Initialize video and output writer for Video 1
video_1_path = "/content/Traffic_Laramie_1.mp4"
output_1_path = "/content/Traffic_1_Counting_Output.mp4"
video_1, fps_1, frame_width_1, frame_height_1, total_frames_1 = initialize_video(video_1_path)
output_video_1 = initialize_video_writer(output_1_path, fps_1, frame_width_1, frame_height_1)

# Process video 1
process_video(video_1, output_video_1, fps_1, total_frames_1)

Video loaded: /content/Traffic_Laramie_1.mp4
FPS: 25, Width: 1040, Height: 600, Total Frames: 4448
Output video writer initialized at `/content/Traffic_1_Counting_Output.mp4`
Initial frame read successfully.
Starting video processing...
Car counted! Total count: 1
Car counted! Total count: 2
Car counted! Total count: 3
Car counted! Total count: 4
Car counted! Total count: 5
Car counted! Total count: 6
Video processing complete....

----- Final Results -----
Total cars counted passing to the city center: 6
Cars per minute: 2.02
Video processing complete.


In [ ]:
# Initialize video and output writer for Video 2
video_2_path = "/content/Traffic_Laramie_2.mp4"
output_2_path = "/content/Traffic_2_Counting_Output.mp4"
video_2, fps_2, frame_width_2, frame_height_2, total_frames_2 = initialize_video(video_2_path)
output_video_2 = initialize_video_writer(output_2_path, fps_2, frame_width_2, frame_height_2)

# Process video 2
process_video(video_2, output_video_2, fps_2, total_frames_2)

Video loaded: /content/Traffic_Laramie_2.mp4
FPS: 25, Width: 1040, Height: 600, Total Frames: 2642
Output video writer initialized at `/content/Traffic_2_Counting_Output.mp4`
Initial frame read successfully.
Starting video processing...
Car counted! Total count: 1
Car counted! Total count: 2
Car counted! Total count: 3
Car counted! Total count: 4
Video processing complete....

----- Final Results -----
Total cars counted passing to the city center: 4
Cars per minute: 2.27
Video processing complete.
